## Chapter 7. 합성곱 신경망(CNN)
* 합성곱 신경망(convolutional neural network, CNN) : 이미지 인식과 음성 인식 등 다양한 곳에서 사용됨

#### 7.1 전체구조
* 합성곱 계층
* 풀링 계층 
* 완전연결(fully-connected,전결합) : 인접하는 계층의 모든 뉴런과 결합되어 있음 
    * ex.Affine 계층 
    * 구조 :  입력 -> Affine -> ReLU -> Affine -> ReLU -> Affine -> ReLU -> Affine -> Softmax -> 출력
* CNN 구조
    * Conv(합성곱 계층)
    * Pooling(풀링 계층)
    * 구조 : 입력 -> Conv -> ReLU -> Pooling -> Conv -> ReLU -> Pooling -> Conv -> ReLU -> Affine -> ReLU -> Affine -> Softmax -> 출력
        * Affine-ReLU 연결이 Conv-ReLU-(Pooling)으로 바뀌었다고 볼 수 있음
        * 출력에 가까운 층에서는 기존의 Affine-ReLU 구성을 사용할 수 있음
        * 마지막 출력 계층에서는 Affine-Softmax 구성을 그대로 사용 
    
#### 7.2 합성곱 계층
* CNN과 FC(Fully-connected)의 차이 : CNN은 각 계층 사이에 3차원 데이터같이 입체적인 데이터가 흐름

* 완전연결 계층(FC)
    * 데이터의 형상이 무시된다
        * 이미지의 경우 세로,가로,색상으로 구성된 3차원 데이터
            * FC에 이미지를 입력할 때는 3차원 데이터를 1차원 데이터로 평탄화 해주어야 함
        * 3차원 형상일 때 공간적인 정보(가까운 픽셀끼리의 유사성 등)를 가지고 있지만 FC에서 형상을 무시하면 해당정보를 잃게 됨
    
* 합성곱 계층
    * 데이터의 형상을 유지한다
        * 이미지의 경우 3차원 데이터를 그대로 입력받고, 3차원 데이터로 전달함
        * 이미지를 더 잘 이해할 가능성이 있음 
    * 특징 맵(feature map) : 합성곱 계층의 '입출력 데이터'
        * 입력 특징 맵(input feature map) : 합성곱 계층의 입력데이터 
        * 출력 특징 맵(output feature map) : 합성곱 계층의 출력 데이터 
    * 합성곱 연산을 처리
        * ![nn](fused_multiply_add.jpeg)
        * 합성곱 연산에서의 편향은 필터를 적용한 원소에 broadcast적용
        * ![nn](add_bia.jpeg)

* 패딩(padding) : 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(ex. 0)으로 채우는 행위
    * 패딩 : 1 -> 폭이 1인 패딩을 적용
    * 패딩을 사용하는 이유 : 출력 데이터의 크기를 조정할 수 있음 
        * 입력데이터가 (4,4)일 때, (3,3)필터를 적용하면 출력 데이터는 (2,2)
        * 폭이 1인 패딩을 적용할 경우 출력데이터는 (4,4)
        * ![nn](padding.jpeg)
        
* 스트라이드(stride) : 필터를 적용하는 위치의 간격
    * 지금까지 모든 예시에는 스트라이드가 1 
    * ex. 스트라이드가 2라면 필터를 적용하는 윈도우가 두칸씩 이동
    * 스트라이드를 키우면 출력
        * ![nn](stride.jpeg)


* 스트라이드와  패딩, 데이터 크기의 관계식 
    * $(H, W)$, 필터 크기 : $(FH, FW)$, 출력 크기 : $(OH, OW)$, 패딩 : $P$, 스트라이드 : $S$
    * 출력 크기 $(OH,OW)$ (정수로 나누어 떨어져야함)
        * $OH = \frac{H+2P-FH}{S}+1$
        * $OW = \frac{W+2P-FW}{S}+1$
        
        
* 3차원 데이터의 합성곱 연산 
    * 2차원 데이터와 비교했을 때 길이 방향(채널 방향)으로 특징 맵이 늘어남
    * 채널 마다 합성곱 연산을 수행한 뒤 결과를 더해서 하나의 출력을 얻음
        * 입력 데이터의 채널 수와 필터 채널의 수가 같아야 함
        * 필터 자체의 크기는 원하는 값으로 설정 할 수 있음
            * 모든 채널의 필터의 크기가 같아야 함
        * ![nn](3dim.jpeg)
        
        
* 블록으로 생각하기 
    * 데이터와 필터를 직육면체 블록이라고 생각하면 쉬움
    * 3차원 데이터를 배열로 표현 : (C(channel), H(height), W(width))
        * 필터의 경우 : (C, FH, FW)
        * 출력데이터가 한 개의 채널을 가짐 
        * 입력데이터(C,H,W) $\circledast$ 필터(C,FH,FW)$\to$ 출력데이터(1,OH,OW) 
    * CNN의 처리흐름 : 출력을 다수의 채널으로 내보내려면? 필터를 다수 사용 
        * 필터 : (FN, C, FH, FW) = (출력 채널 수, 입력 채널수, 높이, 너비)
        * 입력데이터(C,H,W) $\circledast$ 필터(FN,C,FH,FW)$\to$ 출력 데이터1(FN,OH,OW) + 편향(FN,1,1)$\to$출력 데이터2(FN,OH,OW)
        * ![nn](CNN_flow.jpeg)
    
* 배치 처리
    * 신경망과 마찬가지로 합성곱 연산에서도 배치 처리를 지원 가능
    * 각 계층을 흐르는 데이터의 차원을 하나 늘림 -> 4차원(데이터 수(N), 채널 수(C), 높이(H), 너비(W))
        * 배치용 차원 N을 선두에 추가
        * 신경망에 4차원 데이터가 하나 흐를 때 마다 데이터 N개에 대한 합성곱 연산이 이루어짐(N회 분의 처리를 한번에 수행) 
        * ![nn](CNN_batchflow.jpeg)
    
#### 7.3 풀링 계층
* 풀링(Pooling) : 세로 * 가로 방향의 공간을 줄이는 연산
* 최대 풀링(max pooling) : 최댓값을 구하는 연산 
    * ex. 2 x 2 최대풀링을 스트라이드 2로 처리
        * 2 x 2 : 대상 영역의 크기, 대상 영역에서 가장 큰 원소 하나를 꺼냄(최대 풀링) 
        * 스트라이드 2 : 2칸 간격 이동 
    * ![nn](maxpooling.jpeg)
* 평균 풀링(average pooling) : 대상영역에서의 평균을 계산
    * 이미지 인식 분야에서는 주로 최대 풀링을 사용 

* 풀링 계층의 특징
    * 학습해야 할 매개변수가 없다
        * 풀링 계층은 합성곱 계층과 달리 학습해야할 매개변수가 없음(최댓값이나 평균을 취하는 처리일뿐)
    * 채널 수가 변하지 않는다 
        * 입력데이터의 채널 수 그대로 출력 데이터로 내보냄 
    * 입력의 변화에 영향을 적게 받는다(강건하다)
        * 입력데이터가 조금 변해도 풀링의 결과는 잘 변하지 않음(ex.데이터가 1만큼 어긋나는 경우)

#### 7.4 합성곱/풀링 계층 구현하기
* 4차원 배열

In [17]:
import numpy as np

In [18]:
# CNN에서 계층 사이를 흐르는 데이터는 4차원 
# 데이터의 형상 : (10,1,28,28) -> 데이터 개수 10, 채널 1, 높이 28, 너비 28
x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [19]:
# 첫번째 데이터에 접근
x[0]

array([[[7.30280974e-01, 2.17959701e-01, 9.83082385e-01, 2.92311243e-01,
         5.57761044e-01, 1.28799532e-01, 3.68336746e-01, 6.07472428e-01,
         4.69767720e-01, 7.55653921e-01, 6.97370793e-01, 9.68301015e-01,
         8.90406867e-01, 4.17040371e-01, 8.69954188e-01, 3.35625438e-01,
         2.39128042e-01, 2.08113787e-01, 3.81619193e-01, 1.49287851e-01,
         4.64198152e-01, 9.26854147e-01, 2.31725197e-01, 6.25815959e-01,
         9.53228208e-01, 5.05664443e-01, 4.04076944e-01, 3.91336179e-01],
        [5.20377890e-01, 3.46637196e-01, 9.97238846e-01, 4.49478044e-01,
         7.59409898e-01, 1.50929307e-02, 1.59017453e-01, 3.39330614e-01,
         8.13257069e-01, 5.04063124e-02, 5.33781728e-01, 9.81371160e-01,
         9.54182975e-01, 3.10325915e-01, 4.76350353e-01, 9.95443880e-01,
         5.45669826e-03, 4.91599231e-01, 9.66426036e-01, 6.38085788e-01,
         2.76715765e-01, 1.95658171e-01, 9.56678056e-01, 9.20478791e-01,
         2.89894920e-01, 5.21001061e-01, 7.1817336

In [20]:
# 두번째 데이터에 접근
x[1]

array([[[5.47246914e-01, 5.67136586e-01, 9.36623385e-01, 1.66616709e-02,
         7.70144946e-01, 4.14467406e-01, 8.43105597e-01, 8.09422590e-01,
         2.82304571e-01, 6.87633118e-01, 6.08616947e-01, 6.56124579e-02,
         3.57219472e-01, 9.98228149e-01, 8.86240324e-01, 3.12072385e-01,
         2.69662644e-03, 7.35814087e-01, 4.37728008e-02, 2.66286109e-01,
         2.49709200e-01, 8.56781703e-01, 8.52285135e-01, 8.17078607e-01,
         8.75288760e-02, 6.56530542e-01, 3.06781632e-01, 6.57631087e-01],
        [2.85670592e-01, 8.11306646e-01, 3.70474231e-01, 7.82141410e-01,
         7.74835214e-01, 4.41302895e-01, 7.80921804e-01, 2.36959896e-01,
         6.73223157e-01, 2.52261706e-01, 9.73067566e-01, 2.84586529e-01,
         6.79129019e-01, 5.30751247e-01, 3.41895503e-01, 1.37659814e-01,
         3.57475018e-01, 9.82057364e-01, 6.56312216e-01, 2.28681360e-01,
         6.51647946e-01, 6.99352233e-02, 9.09505205e-01, 1.95505171e-01,
         5.16866445e-01, 5.32261453e-01, 5.8404057

In [21]:
# 첫번쨰 데이터의 첫 채널의 공간데이터에 접근
x[0,0]

array([[7.30280974e-01, 2.17959701e-01, 9.83082385e-01, 2.92311243e-01,
        5.57761044e-01, 1.28799532e-01, 3.68336746e-01, 6.07472428e-01,
        4.69767720e-01, 7.55653921e-01, 6.97370793e-01, 9.68301015e-01,
        8.90406867e-01, 4.17040371e-01, 8.69954188e-01, 3.35625438e-01,
        2.39128042e-01, 2.08113787e-01, 3.81619193e-01, 1.49287851e-01,
        4.64198152e-01, 9.26854147e-01, 2.31725197e-01, 6.25815959e-01,
        9.53228208e-01, 5.05664443e-01, 4.04076944e-01, 3.91336179e-01],
       [5.20377890e-01, 3.46637196e-01, 9.97238846e-01, 4.49478044e-01,
        7.59409898e-01, 1.50929307e-02, 1.59017453e-01, 3.39330614e-01,
        8.13257069e-01, 5.04063124e-02, 5.33781728e-01, 9.81371160e-01,
        9.54182975e-01, 3.10325915e-01, 4.76350353e-01, 9.95443880e-01,
        5.45669826e-03, 4.91599231e-01, 9.66426036e-01, 6.38085788e-01,
        2.76715765e-01, 1.95658171e-01, 9.56678056e-01, 9.20478791e-01,
        2.89894920e-01, 5.21001061e-01, 7.18173365e-01, 4.78841

* im2col로 데이터 전개하기 : '트릭'을 사용해서 4차원 데이터를 다룬 연산의 구현을 쉽게 만들어줌
    * 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는 함수
        * 3차원 입력 데이터에 im2Col을 적용하면 2차원 행렬로 바뀜(배치 안의 데이터 수까지 포함한 4차원 데이터를 2차원으로 변환)
            * 3차원 블록을 한 줄로 늘어 놓음(image to column)
            * ![nn](im2col.jpeg)
        * 그림 에서는 보기 좋게 스트라이드를 잡아 필터의 적용 영역이 겹치지 않도록 했지만, 실제에서는 영역이 겹치는 경우가 대부분
            * im2col을 사용해 구현하면 메모리를 더 많이 소비하는 단점이 존재하지만, 문제를 행렬 계산으로 만들면 선형 대수 라이브러리를 활용해 효율을 높일 수 있음 
    * im2col로 입력 데이터를 전개한 다음 합성곱 계층의 필터(가중치)를 1열로 전개, 두 행렬의 곱을 계산
        * FC의 Affine계층에서 한 것과 거의 유사
        * 출력한 결과는 2차원 행렬 -> CNN은 데이터를 4차원 배열로 저장하므로 4차원으로 reshape해줌 
        * ![nn](im2col_detail.jpeg)
        
* 합성곱 계층 구현하기
    * `im2col(input_data, filter_h, filter_w, stride=1, pad=0)`
        * input_data : (N,C,H,W)로 이루어진 4차원 입력데이터
        * filter_h : 필터의 높이
        * filter_w : 필터의 너비
        * stride : 스트라이드
        * pad : 패딩





In [22]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

# import sys, os
# sys.path.append(os.pardir)
# from common.util import im2col

x1 = np.random.rand(1,3,7,7) # 4차원 랜덤 행렬 # 데이터 : 1개 , 채널 : 3개, 높이 x 너비 : 7x7
col1 = im2col(x1,5,5,stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad =0)
print(col2.shape)

# 입력 데이터의 개수가 많은 것의 출력값은 크기가 더 커진 형태로 반환됨

(9, 75)
(90, 75)


In [23]:
# 합성곱 계층 구현
class Convolution :
    def __init__(self, W, b, stride=1, pad=0) :
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x) :
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개 
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

* 풀링 계층 구현하기
    * 풀링의 경우 : 채널 쪽이 독립적이라는 점이 합성곱 계층 때와 다름
    * 풀링 계층 forward 처리 흐름
    * ![nn](poolingfoward.jpeg)

In [26]:
# 풀링 forward

class Pooling :
    def __init__(self, pool_h, pool_w, stride =1, pad =0) :
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x) :
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 입력 데이터 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 행별 최댓값(2), 
        out = np.max(col, axis = 1)
        
        # 적절한 모양으로 변환, 성형 (3) 
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out
    
    # backward 처리는 ReLU와 유사 (max의 역전파) 

#### 7.5 CNN 구현하기

* SimpleConvNet : Convolution - relu - pooling - affine - relu - afffine - softmax
    * `__init__`
        * input_dim : 입력 데이터(채널 수, 높이, 너비)의 차원
        * conv_param : 합성곱 계층의 하이퍼 파라미터(딕셔너리) - ex.{'filter_num' : 30, 'filter_size' : 5, 'pad': 0. 'stride' : 1}
            * filter_num : 필터 수
            * filter_size : 필터 크기 
            * stride : 스트라이드
            * pad : 패딩
        * hidden_size : 은닉층(완전연결)의 뉴런 수
        * output_size : 출력층(완전연결)의 뉴런 수
        * weight_init_std : 초기화 때의 가중치 표준편차
    

In [27]:
class SimpleConvNet :
    def __init__(self, input_dim = (1, 28, 28),conv_param = {'filter_num' : 30, 'filter_size' : 5, 'pad' : 0, ' stride':1},
                hidden_size = 100, output_size = 10, weight_init_std = 0.01) :
        # 하이퍼 파라미터를 딕셔너리에서 꺼냄
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        # 합성곱 계층의 출력 크기를 계산 
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2)*(conv_output_size/2))
        

#### 7.6 CNN 시각화 하기
* 1번째 층의 가중치 시각화 하기
* 층 깊이에 따른 추출 정보 변화



#### 7.7 대표적인 CNN
* LeNet
* AlexNet

#### 7.8 정리 